# KfServing Logger demo

We create a message dumper KNaive service to print out CloudEvents it receives:

In [1]:
!pygmentize message-dumper.yaml

apiVersion: serving.knative.dev/v1alpha1
kind: Service
metadata:
  name: message-dumper
spec:
  template:
    spec:
      containers:
      - image: gcr.io/knative-releases/github.com/knative/eventing-sources/cmd/event_display


In [2]:
!kubectl apply -f message-dumper.yaml

service.serving.knative.dev/message-dumper created


Create a SkLearn model with associated logger to push events to the message logger URL

In [4]:
!pygmentize sklearn-logging.yaml

apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "sklearn-iris"
spec:
  default:
    predictor:
      minReplicas: 1      
      logger:
        url: http://message-dumper.default/
        mode: all
      sklearn:
        storageUri: "gs://kfserving-samples/models/sklearn/iris"
        resources:
          requests:
            cpu: 0.1
                                  


In [6]:
!kubectl apply -f sklearn-logging.yaml

inferenceservice.serving.kubeflow.org/sklearn-iris created


In [7]:
CLUSTER_IPS=!(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
CLUSTER_IP=CLUSTER_IPS[0]
print(CLUSTER_IP)

35.204.158.239


In [8]:
SERVICE_HOSTNAMES=!(kubectl get inferenceservice sklearn-iris -o jsonpath='{.status.url}' | cut -d "/" -f 3)
SERVICE_HOSTNAME=SERVICE_HOSTNAMES[0]
print(SERVICE_HOSTNAME)

sklearn-iris.default.example.com


In [16]:
import requests
def predict(X, name, svc_hostname, cluster_ip):
    formData = {
    'instances': X
    }
    headers = {}
    headers["Host"] = svc_hostname
    res = requests.post('http://'+cluster_ip+'/v1/models/'+name+':predict', json=formData, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        print("Failed with ",res.status_code)
        return []

In [20]:
predict([[6.8,  2.8,  4.8,  1.4]],"sklearn-iris",SERVICE_HOSTNAME,CLUSTER_IP)

{'predictions': [1]}

In [21]:
!kubectl logs  $(kubectl get pod -l serving.knative.dev/configuration=message-dumper -o jsonpath='{.items[0].metadata.name}') user-container

☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 0.2
  type: org.kubeflow.serving.inference.response
  source: http://localhost:8080/
  id: cd767a44-54dd-4bd4-b115-6d26514d31db
  time: 2019-11-03T20:07:22.912383978Z
  contenttype: application/json; charset=UTF-8
Data,
  {
    "predictions": [
      1
    ]
  }
☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 0.2
  type: org.kubeflow.serving.inference.request
  source: http://localhost:8080/
  id: cd767a44-54dd-4bd4-b115-6d26514d31db
  time: 2019-11-03T20:07:22.895590001Z
  contenttype: application/json
Data,
  {
    "instances": [
      [
        6.8,
        2.8,
        4.8,
        1.4
      ]
    ]
  }
☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 0.2
  type: org.kubeflow.serving.inference.request
  source: http://localhost:8080/
  id: cf1bc6e7-725a-4354-9e02-a8a6a81aba91
  time: 2019-11-03T20:08:24.494820358Z
  contenttype: application/json
Data,
  {


In [22]:
!kubectl delete -f sklearn-logging.yaml

inferenceservice.serving.kubeflow.org "sklearn-iris" deleted


In [23]:
!kubectl delete -f message-dumper.yaml

service.serving.knative.dev "message-dumper" deleted
